In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import folium
from IPython.display import IFrame   
from IPython.core.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Paths
RESPONSES_19 = '../input/kaggle-survey-2019/multiple_choice_responses.csv'
RESPONSES_18 = '../input/kaggle-survey-2018/multipleChoiceResponses.csv'
RESPONSES_17 = '../input/kaggle-survey-2017/multipleChoiceResponses.csv'
WORLDBANK_CLASS = '../input/world-bank-country-and-lending-groups/worldbank_classification.csv'
WORLD_COUNTRIES_JSON = '../input/2019-kaggle-survey-challenge-images/world-countries.json'

# Colours and formatting
DEFAULT_COLOUR = '#0075c4' 
HIGH_COLOUR = '#2c63c9'
UP_MID_COLOUR = '#7d2e68'
LOW_MID_COLOUR = '#d72638'
ALPHA=0.85
sns.set_style("whitegrid")
INCOME_GROUP_ORDER = ['Lower mid', 'Upper mid', 'High']

"""
The following encodings are mainly used to control ordering of categories on plots 
and in some cases to assign more readable labels.
"""
# ordinal encodings
encoding_dict = {}
encoding_dict['Q1'] =  { # Age
    '18-21' : 0, '22-24' : 1, '25-29' : 2, '30-34' : 3, '35-39' : 4, 
    '40-44' : 5, '45-49' : 6, '50-54' : 7, '55-59' : 8, '60-69' : 9, 
    '70+' : 10,
}
encoding_dict['Q4'] = { # Education
    'I prefer not to answer': 0, 
    'Highschool or lower' : 1,
    'Some university': 2, 
    'Bachelor’s degree' : 3,
    'Master’s degree' : 4, 
    'Doctoral degree' : 5,
    'Professional degree' : 6, 
}
encoding_dict['Q6'] = { # Company size
    '0-49 employees': 0, 
    '50-249 employees' : 1, 
    '250-999 employees': 2, 
    '1000-9,999 employees' : 3,
    '> 10,000 employees' : 4,
}
encoding_dict['Q7'] = { # Number of individuals responsible for data science
    '0' : 0,
    '1-2': 1,
    '3-4': 2,
    '5-9': 3,
    '10-14': 4,
    '15-19': 5,
    '20+': 6
}
encoding_dict['Q8'] = { # Use of ML methods
    "Don't know": 0,
    'None': 1,
    'Exploring': 2,
    'Insights only': 3,
    'Productive < 2 years': 4, 
    'Productive > 2 years': 5

}
encoding_dict['Q11'] = { # Spending on ML/Cloud products
    '$0 (USD)': 0,
    '$1-$99': 1,
    '$100-$999': 2,
    '$1000-$9,999': 3,
    '$10,000-$99,999': 4,
    '> $100,000 ($USD)': 5
}
encoding_dict['Q10'] =  { # Salary
    '$0-999' : 0, '1,000-1,999' : 1, '2,000-2,999' : 2, '3,000-3,999' : 3, '4,000-4,999' : 4,
    '5,000-7,499' : 5, '7,500-9,999' : 6, '10,000-14,999' : 7, '15,000-19,999' : 8, '20,000-24,999' : 9,
    '25,000-29,999' : 10, '30,000-39,999' : 11, '40,000-49,999' : 12, '50,000-59,999' : 13, '60,000-69,999' : 14,
    '70,000-79,999' : 15, '80,000-89,999' : 16, '90,000-99,999' : 17, '100,000-124,999' : 18, '125,000-149,999' : 19,
    '150,000-199,999' : 20,    '200,000-249,999' : 21, '250,000-299,999' : 22, '300,000-500,000' : 23, '> $500,000' : 24,
}
encoding_dict['Q15'] =  { # Coding experience
    'None' : 0, '< 1 years' : 1, '1-2 years' : 2, '3-5 years' : 3, '5-10 years' : 4,
    '10-20 years' : 5, '20+ years' : 6,
}
encoding_dict['Q22'] =  { # TPU use
    'Never': 0, 
    'Once': 1,
    '2-5 times': 2, 
    '6-24 times': 3,
    '> 25 times': 4
}
encoding_dict['Q23'] =  { # ML experience
    '< 1 years' : 0, '1-2 years' : 1, '2-3 years' : 2, '3-4 years' : 3, '4-5 years' : 4, 
    '5-10 years' : 5, '10-15 years' : 6, '15-20 years' : 7, '20+ years' : 8,
}
# nominal encodings
encoding_dict['Q2'] = { # gender
    'Male': 'male',
    'Female': 'female',
    'Prefer not to say': 'prefer not to say',
    'Prefer to self-describe': 'self-describe',
}
encoding_dict['Q5'] = { # title
    'Not employed': 'Not employed', 
    'Student': 'Student',
    'Product/Project Manager': 'Product/Project Manager',
    'Business Analyst': 'Business Analyst',
    'Data Analyst': 'Data Analyst', 
    'Data Engineer': 'Data Engineer', 
    'DBA/Database Engineer': 'DBA/Database Engineer', 
    'Software Engineer': 'Software Engineer', 
    'Statistician': 'Statistician',
    'Data Scientist': 'Data Scientist', 
    'Research Scientist': 'Research Scientist', 
    'Other': 'Other', 
}
encoding_dict['Q9'] =  { # work activities 
    'Q9_Part_7': 'None',
    'Q9_Part_1': 'Data analysis',
    'Q9_Part_2': 'Data Infrastructure',
    'Q9_Part_3': 'Build ML Prototypes',
    'Q9_Part_4': 'Build/Run ML Services',
    'Q9_Part_5': 'Experimentation ML Models',
    'Q9_Part_6': 'ML Research',
    'Q9_Part_8': 'Other',
}
encoding_dict['Q12'] =  { # media 
    'Q12_Part_11': 'None',
    'Q12_Part_1': 'Twitter',
    'Q12_Part_2': 'Hacker News',
    'Q12_Part_3': 'Reddit',
    'Q12_Part_4': 'Kaggle',
    'Q12_Part_5': 'course forums',
    'Q12_Part_6': 'YouTube',
    'Q12_Part_7': 'podcasts',
    'Q12_Part_8': 'blogs',
    'Q12_Part_9': 'journals',
    'Q12_Part_10': 'Slack',
    'Q12_Part_12': 'Other',
}
encoding_dict['Q13'] =  { # DS course platforms 
    'Q13_Part_11': 'None',
    'Q13_Part_1': 'Udacity',
    'Q13_Part_2': 'Coursera',
    'Q13_Part_3': 'edX',
    'Q13_Part_4': 'DataCamp',
    'Q13_Part_5': 'DataQuest',
    'Q13_Part_6': 'Kaggle Courses',
    'Q13_Part_7': 'Fast.ai',
    'Q13_Part_8': 'Udemy',
    'Q13_Part_9': 'LinkedIn Learning',
    'Q13_Part_10': 'University Courses',
    'Q13_Part_12': 'Other',
}
encoding_dict['Q14'] =  { # Tools (software)
    'None': 'None',
    'Basic statistical software (Microsoft Excel, Google Sheets, etc.)': 'basic stats',
    'Advanced statistical software (SPSS, SAS, etc.)': 'advanced stats',
    'Local development environments (RStudio, JupyterLab, etc.)': 'local dev env',
    'Business intelligence software (Salesforce, Tableau, Spotfire, etc.)': 'BI software',
    'Cloud-based data software & APIs (AWS, GCP, Azure, etc.)': 'Cloud-based'   
}
encoding_dict['Q16'] =  { # IDE
    'Q16_Part_11': 'None',
    'Q16_Part_1': 'Jupyter',
    'Q16_Part_2': 'RStudio',
    'Q16_Part_3': 'PyCharm',
    'Q16_Part_4': 'Atom',
    'Q16_Part_5': 'MATLAB',
    'Q16_Part_6': 'Visual Studio',
    'Q16_Part_7': 'Spyder',
    'Q16_Part_8': 'Vim / Emacs',
    'Q16_Part_9': 'Notepad++',
    'Q16_Part_10': 'Sublime Text',
    'Q16_Part_12': 'Other',
}
encoding_dict['Q17'] =  { # Hosted Notebooks
    'Q17_Part_11': 'None',
    'Q17_Part_1': 'Kaggle Notebooks',
    'Q17_Part_2': 'Google Colab',
    'Q17_Part_3': 'Microsoft Azure',
    'Q17_Part_4': 'Google Cloud',
    'Q17_Part_5': 'Paperspace / Graient',
    'Q17_Part_6': 'FloydHub',
    'Q17_Part_7': 'Binder / JupyterHub',
    'Q17_Part_8': 'IBM Watson',
    'Q17_Part_9': 'code Ocean',
    'Q17_Part_10': 'AWS Products',
    'Q17_Part_12': 'Other',
}
encoding_dict['Q18'] =  { # Languages
    'Q18_Part_11': 'None',
    'Q18_Part_1': 'Python',
    'Q18_Part_2': 'R',
    'Q18_Part_3': 'SQL',
    'Q18_Part_4': 'C',
    'Q18_Part_5': 'C++',
    'Q18_Part_6': 'Java',
    'Q18_Part_7': 'Javascript',
    'Q18_Part_8': 'TypeScript',
    'Q18_Part_9': 'Bash',
    'Q18_Part_10': 'MATLAB',
    'Q18_Part_12': 'Other',
}
encoding_dict['Q20'] =  { # Data viz libraries
    'Q20_Part_11': 'None',
    'Q20_Part_1': 'Ggplot',
    'Q20_Part_2': 'Matplotlib',
    'Q20_Part_3': 'Altair',
    'Q20_Part_4': 'Shiny',
    'Q20_Part_5': 'D3.js',
    'Q20_Part_6': 'Plotly',
    'Q20_Part_7': 'Bokeh',
    'Q20_Part_8': 'Seaborn',
    'Q20_Part_9': 'Geoplotlib',
    'Q20_Part_10': 'Leaflet / Folium',
    'Q20_Part_12': 'Other',
}
encoding_dict['Q21'] =  { # Specialised Hardware use
    'Q21_Part_4': "None / Don't know", 
    'Q21_Part_1': 'CPUs',
    'Q21_Part_2': 'GPUs',
    'Q21_Part_3': 'TPUs',
    'Q21_Part_5': 'Other',
}
encoding_dict['Q24'] =  { # ML Algorithms
    'Q24_Part_11': 'None',
    'Q24_Part_1': 'Logistic Regression',
    'Q24_Part_2': 'Decision Trees',
    'Q24_Part_3': 'GBMs',
    'Q24_Part_4': 'Bayesian',
    'Q24_Part_5': 'Evolutionary',
    'Q24_Part_6': 'NNs',
    'Q24_Part_7': 'CNNs',
    'Q24_Part_8': 'GANs',
    'Q24_Part_9': 'RNNs',
    'Q24_Part_10': 'Transformers',
    'Q24_Part_12': 'Other',
}
encoding_dict['Q25'] =  { # Automated ML Tools
    'Q25_Part_7': 'None',
    'Q25_Part_1': 'data augmentation',
    'Q25_Part_2': 'feature engineering',
    'Q25_Part_3': 'model selection',
    'Q25_Part_4': 'architecture search',
    'Q25_Part_5': 'hyperparameter tuning',
    'Q25_Part_6': 'full ML pipelines',
    'Q25_Part_8': 'Other',
}
encoding_dict['Q26'] =  { # Computer Vision method categories
    'Q26_Part_6': 'None',
    'Q26_Part_1': 'general purpose',
    'Q26_Part_2': 'image segmentation',
    'Q26_Part_3': 'object detection',
    'Q26_Part_4': 'classification ',
    'Q26_Part_5': 'generative networks',
    'Q26_Part_7': 'Other',
}
encoding_dict['Q27'] =  { # NLP method categories
    'Q27_Part_5': 'None',
    'Q27_Part_1': 'word embeddings',
    'Q27_Part_2': 'encoder-decoder models',
    'Q27_Part_3': 'contextualized embeddings',
    'Q27_Part_4': 'transformer models',
    'Q27_Part_6': 'Other',
}
encoding_dict['Q28'] =  { # ML frameworks
    'Q28_Part_11': 'None',
    'Q28_Part_1': 'Scikit-learn',
    'Q28_Part_2': 'Tensorflow',
    'Q28_Part_3': 'Keras',
    'Q28_Part_4': 'RandomForest',
    'Q28_Part_5': 'Xgboost',
    'Q28_Part_6': 'PyTorch',
    'Q28_Part_7': 'Caret',
    'Q28_Part_8': 'LightGBM',
    'Q28_Part_9': 'Spark MLib',
    'Q28_Part_10': 'Fast.ai',
    'Q28_Part_12': 'Other',
}
encoding_dict['Q29'] =  { # Cloud Platforms
    'Q29_Part_11': 'None',
    'Q29_Part_1': 'GCP',
    'Q29_Part_2': 'AWS',
    'Q29_Part_3': 'Azure',
    'Q29_Part_4': 'IBM',
    'Q29_Part_5': 'Alibaba',
    'Q29_Part_6': 'Salesforce',
    'Q29_Part_7': 'Oracle',
    'Q29_Part_8': 'SAP',
    'Q29_Part_9': 'VMware',
    'Q29_Part_10': 'Red Hat',
    'Q29_Part_12': 'Other',
}
encoding_dict['Q30'] =  { # Cloud Products
    'Q30_Part_11': 'None',
    'Q30_Part_1': 'AWS EC2',
    'Q30_Part_2': 'Google Compute Engine',
    'Q30_Part_3': 'AWS Lambda',
    'Q30_Part_4': 'Azure VMs',
    'Q30_Part_5': 'Google App Engine',
    'Q30_Part_6': 'Google Cloud Functions',
    'Q30_Part_7': 'AWS Elastic Beanstalk',
    'Q30_Part_8': 'Google Kubernetes Engine',
    'Q30_Part_9': 'AWS Batch',
    'Q30_Part_10': 'Azure Container Service',
    'Q30_Part_12': 'Other',
}
encoding_dict['Q31'] =  { # Big Data Analytics Products
    'Q31_Part_11': 'None',
    'Q31_Part_1': 'Google BigQuery',
    'Q31_Part_2': 'AWS Redshift',
    'Q31_Part_3': 'Databricks',
    'Q31_Part_4': 'AWS Elastic MapReduce',
    'Q31_Part_5': 'Teradata',
    'Q31_Part_6': 'Microsoft Analysis Services',
    'Q31_Part_7': 'Google Cloud Dataflow',
    'Q31_Part_8': 'AWS Athena',
    'Q31_Part_9': 'AWS Kinesis',
    'Q31_Part_10': 'Google Cloud Pub/Sub',
    'Q31_Part_12': 'Other',
}
encoding_dict['Q32'] =  { # ML Products
    'Q32_Part_11': 'None',
    'Q32_Part_1': 'SAS',
    'Q32_Part_2': 'Cloudera',
    'Q32_Part_3': 'Azure ML Studio',
    'Q32_Part_4': 'Google Cloud ML Engine',
    'Q32_Part_5': 'Google Cloud Vision',
    'Q32_Part_6': 'Google Cloud Speech-to-Text',
    'Q32_Part_7': 'Google Cloud Natural Language',
    'Q32_Part_8': 'RapidMiner',
    'Q32_Part_9': 'Google Cloud Translation',
    'Q32_Part_10': 'Amazon SageMaker',
    'Q32_Part_12': 'Other',
}
encoding_dict['Q33'] =  { # Automated ML Tools
    'Q33_Part_11': 'None',
    'Q33_Part_1': 'Google AutoML',
    'Q33_Part_2': 'H2O Driverless AI',
    'Q33_Part_3': 'Databricks AutoML',
    'Q33_Part_4': 'DataRobot AutoML',
    'Q33_Part_5': 'Tpot',
    'Q33_Part_6': 'Auto-Keras',
    'Q33_Part_7': 'Auto-Sklearn',
    'Q33_Part_8': 'Auto_ml',
    'Q33_Part_9': 'Xcessive',
    'Q33_Part_10': 'MLbox',
    'Q33_Part_12': 'Other',
}
encoding_dict['Q34'] =  { # Relational DB Products
    'Q34_Part_11': 'None',
    'Q34_Part_1': 'MySQL',
    'Q34_Part_2': 'PostgresSQL',
    'Q34_Part_3': 'SQLite',
    'Q34_Part_4': 'Microsoft SQL Server',
    'Q34_Part_5': 'Oracle',
    'Q34_Part_6': 'Microsoft Access',
    'Q34_Part_7': 'AWS Relational DB Service',
    'Q34_Part_8': 'AWS DynamoDB',
    'Q34_Part_9': 'Azure SQL DB',
    'Q34_Part_10': 'Google Cloud SQL',
    'Q34_Part_12': 'Other',
}

# The Gap between Rich and Poor in the AI Revolution
**Tania Jacob - 29.11.2019**
***

The year is 2019 and tech, lead by data science and biotech, is moving at breakneck pace. The computing power needed to train AIs is rising seven times faster than ever before [1]. The future of work is a major topic of discussion because AI will no doubt change the nature of jobs, if not eliminate many of them. Biases have crept into our AI systems, reflecting the biases of their creators. 

CRISPR gene-editing technology already allows us to play around with genomes [2], meaning that human upgrades and designer children will soon be reality. Just a few weeks ago it was reported that the first genetic tests, including intelligence scores, done on embryos before they are selected for IVF implantation are now available [3]. How long will it be before the rich start using this technology to ensure all their offspring have as high intelligence as possible? How long before any number of other characteristics start being selected for instead of left to chance? And what will this mean for those who cannot afford such biotech? 

In the previous century, the rich had a stake in fixing the problems of the poor because they were economically and militarily vital. This may no longer be the case with AI coming of age. "Not just entire classes, but entire countries and continents might become irrelevant." (Harari, 2018, p.93) [4]

At the same time, we hear many positive stories about efforts to bring AI to poorer regions. In 2013 IBM Research opened its first African office in Nairobi. Deep Learning Indaba, formed in 2017, now has chapters in 27 of Africa's 54 countries [5]. Google announced their first research center in Africa (Ghana) in 2018 [6]. And there have been increasing numbers of AI conferences held in Africa to circumvent the difficulties (visas, costs, etc.) people from these countries often face to attend such conferences. Indeed, last year's Kaggle Survey Challenge winner gave a very positive report on the state of AI in Africa [7]. 

No doubt AI has the potential to either leapfrog or cripple the growth of developing nations. So where are we really with the state of data science and ML in rich vs. poor countries? This is the subject of the story I wish to tell, about inequality between countries, with a particular focus on low income countries.

# Data and Methodology
***
The report is based on data from Kaggle's third annual survey, with responses from 19,717 Kaggle members. Of these responses, only the 18,663 for which a specific country could be associated with the respondent were considered. The complete dataset and information on the survey methodology can be found [here](https://www.kaggle.com/c/kaggle-survey-2019/data).

In order to explore the gap between rich and poor, the respondents can be grouped by countries (Q3) according to income levels. This is done using a list obtained from the World Bank [8][9], where countries are divided into low-, lower-middle-, upper-middle- and high-income economies according to their gross national income (GNI) per capita \*(1). This income divide can be visualised below.

In [ ]:
######################
# Loading functions
######################
def load_worldbank_classification():
    """Load worldbank country classifications."""
    df = pd.read_csv(WORLDBANK_CLASS)
    # Fix Iran
    df.replace({'Iran Islamic Republic of...': 'Iran, Islamic Republic of...'}, inplace=True)
    # Replace income group labels with more readable ones
    df.replace({'High income': 'High'}, inplace=True)
    df.replace({'Upper middle income': 'Upper mid'}, inplace=True)
    df.replace({'Lower middle income': 'Lower mid'}, inplace=True)
    df.replace({'Low income': 'Low'}, inplace=True)
    return df

def load_survey_responses(multiple_choice_responses_file, worldbank_class, encoding=None, country_col='Q3', verbose=False):
    """Load survey reponses"""
    df = pd.read_csv(multiple_choice_responses_file, encoding=encoding)
    # Remove first row (questions)
    df.drop(df.index[0], inplace=True)
    if verbose:
        print(f'\nSurvey Results {multiple_choice_responses_file}')
        print(f'Total questions (including parts and time): {df.shape[1]}')
        print(f'Total responses: {df.shape[0]}')
    # Q2 - Combine other genders into a single group
    df.replace({'Prefer not to say' : 'Other'}, inplace=True)
    df.replace({'Prefer to self-describe' : 'Other'}, inplace=True)
    # Q3 - Shorten some names to match World Bank names and for label convenience
    df.replace({'Iran, Islamic Republic of...' : 'Iran'}, inplace=True)
    df.replace({'United States of America' : 'United States'}, inplace=True)
    df.replace({'United Kingdom of Great Britain and Northern Ireland' : 'United Kingdom'}, inplace=True)   
    # Q3 - Replace some country names (for 2017) to make them consistent with later years
    df.replace({"People 's Republic of China": 'China'}, inplace=True)
    df.replace({'Hong Kong': 'Hong Kong (S.A.R.)'}, inplace=True)
    df.replace({'Republic of China': 'China'}, inplace=True)
    df.replace({'Vietnam': 'Viet Nam'}, inplace=True)
    # Q3 - Merge "Republic of Korea" with "South Korea" into one group
    df.replace({'Republic of Korea': 'South Korea'}, inplace=True)
    # Q4 - Replace ordinal values for the sake of label readability
    df.replace({'No formal education past high school': 'Highschool or lower'}, inplace=True)
    df.replace({'Some college/university study without earning a bachelor’s degree': 'Some university'}, inplace=True)
    # Q8 - Replace ordinal values for the sake of label readability
    df.replace({'I do not know': "Don't know"}, inplace=True)
    df.replace({'No (we do not use ML methods)': 'None'}, inplace=True)
    df.replace({'We are exploring ML methods (and may one day put a model into production)': 'Exploring'}, inplace=True)
    df.replace({'We use ML methods for generating insights (but do not put working models into production)': 'Insights only'}, inplace=True)
    df.replace({'We recently started using ML methods (i.e., models in production for less than 2 years)': 'Productive < 2 years'}, inplace=True)
    df.replace({'We have well established ML methods (i.e., models in production for more than 2 years)': 'Productive > 2 years'}, inplace=True)
    # Q15 - Replace ordinal values for the sake of label readability
    df.replace({'I have never written code': 'None'}, inplace=True)
    # Join to World Bank classifications
    df = df.merge(worldbank_class, left_on=country_col, right_on='Economy')
    if verbose:
        print(f'Number of responses after removing those without country: {df.shape[0]}')
    # Give the dataset a unique index
    df = df.reset_index().rename(columns={df.index.name:'id'}) 
    
    return df

######################
# Plotting Functions
######################
def lineplot_survey_summary(df):
    """Plot counts of three years of survey responses grouped by income group."""
    responses2018 = load_survey_responses(RESPONSES_18, worldbank_class)
    responses2017 = load_survey_responses(RESPONSES_17, worldbank_class, encoding="ISO-8859-1", country_col='Country')

    summary = pd.DataFrame(index=['Lower mid', 'Upper mid', 'High'])
    summary['2019'] = responses.groupby('Income group')['index'].nunique()
    summary['2018'] = responses2018.groupby('Income group')['index'].nunique()
    summary['2017'] = responses2017.groupby('Income group')['index'].nunique()
    summary = summary.T.reset_index().rename(columns={'index':'Year'})
    summary['Year'] = summary['Year'].astype(int)

    fig, ax = plt.subplots(figsize=(12,5))
    sns.pointplot(x='Year', y='Lower mid', data=summary, ax=ax, label='Lower mid income', color=LOW_MID_COLOUR)
    sns.pointplot(x='Year', y='Upper mid', data=summary, ax=ax, label='Upper mid income',color=UP_MID_COLOUR)
    sns.pointplot(x='Year', y='High', data=summary, ax=ax, label='High income', color=HIGH_COLOUR)
    ax.legend(handles=ax.lines[::4], labels=['Lower mid income','Upper mid income','High income'])
    labels = ax.get_xticklabels() 
    ax.set_ylabel('Count')
    plt.title(f'Number of Kaggle Survey Respondents per Year', fontsize=12)

def barplot_income_group_country_counts(df):  
    """Plot counts of countries in each income group."""
    fig = plt.figure(figsize=(15,5))
    ax = plt.subplot2grid((1,3),(0,0))
    l = sns.countplot(df[df['Income group']=='Lower mid']['Q3'].sort_values(), 
                      color=LOW_MID_COLOUR, alpha=ALPHA)
    plt.title(f'Lower-mid income countries', fontsize=12)
    l.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_ylabel('Number of Respondents')
    ax.set_xlabel('')
    
    ax = plt.subplot2grid((1,3),(0,1)) 
    m = sns.countplot(df[df['Income group']=='Upper mid']['Q3'].sort_values(), 
                      color=UP_MID_COLOUR, alpha=ALPHA)
    plt.title(f'Upper-mid income countries', fontsize=12)
    m.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_ylabel('')
    ax.set_xlabel('')
    
    ax = plt.subplot2grid((1,3),(0,2))
    h = sns.countplot(df[df['Income group']=='High']['Q3'].sort_values(), 
                      color=HIGH_COLOUR, alpha=ALPHA)
    plt.title(f'High income countries', fontsize=12)
    h.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_ylabel('')
    ax.set_xlabel('')
    
    plt.tight_layout()
    plt.show()
    
def pieplot_income_group_percentage(df, title, column, xlabel_rotation=45):
    """Plot given column by income group as pie charts."""
    agg = responses.groupby(['Income group'], sort=False)
    col_counts = agg[column].value_counts(normalize=True, sort=False)
    percentage_df = pd.DataFrame(col_counts)
    percentage_df.columns = ['percentage']
    percentage_df.reset_index(inplace=True)
    
    labels = ['Female', 'Male', 'Other']
    LOW_MID_COLOURS = ['#810a1e', '#ef3654', '#F58699']
    UP_MID_COLOURS = ['#34002D', '#7A4C74', '#AF93AB']
    HIGH_COLOURS = ['#10254b', '#2c63c9', '#95B1E4']
    size = 0.5
    shared_pi_formats = {'wedgeprops':dict(width=size, edgecolor='w'),
                     'textprops':dict(color='white', fontsize=10),
                     'autopct':'%.1f %%'}

    fig = plt.figure(figsize=(15,5))
    ax = plt.subplot2grid((1,3),(0,0))
    ax.pie(percentage_df[percentage_df['Income group']=='Lower mid']['percentage'], radius=1,
           colors=LOW_MID_COLOURS, pctdistance=0.75, **shared_pi_formats)
    plt.title(f'{title} - Lower-mid income', fontsize=12)
    ax.legend(loc=(0.8,0.8), labels=labels)
        
    ax = plt.subplot2grid((1,3),(0,1)) 
    ax.pie(percentage_df[percentage_df['Income group']=='Upper mid']['percentage'], radius=1,
           colors=UP_MID_COLOURS, pctdistance=0.75, **shared_pi_formats)
    plt.title(f'{title} - Upper-mid income', fontsize=12)
    ax.legend(loc=(0.8,0.8), labels=labels)
    
    ax = plt.subplot2grid((1,3),(0,2))
    ax.pie(percentage_df[percentage_df['Income group']=='High']['percentage'], radius=1,
           colors=HIGH_COLOURS, pctdistance=0.75, **shared_pi_formats)
    plt.title(f'{title} - High income', fontsize=12)
    ax.legend(loc=(0.8,0.8), labels=labels)
    
    plt.show()
    
def violinplot_income_group(df, title, column):
    """Plot given column by income group as violin plots."""
    ymax = df[column].map(encoding_dict[column]).max()+1
    #fig, ax = plt.subplots()
    plt.figure(figsize=(6, 4))
    ax = sns.violinplot(x=df['Income group'], 
                   y=df[column].map(encoding_dict[column]),
                   #bw=1,
                   order=INCOME_GROUP_ORDER,
                   palette=[LOW_MID_COLOUR, UP_MID_COLOUR, HIGH_COLOUR])
    plt.setp(ax.collections, alpha=ALPHA)
    plt.yticks(np.arange(0, ymax, 1.0))
    ax.set_yticklabels([*encoding_dict[column].keys()])
    ax.set_title(title)
    ax.set_ylabel(column)
    plt.show()
    
def boxplot_income_group(df, title, column):
    """Plot given column by income group as box plots."""
    ymax = df[column].map(encoding_dict[column]).max()+1
    plt.figure(figsize=(6, 4))
    ax = sns.boxplot(x=df['Income group'], 
                   y=df[column].map(encoding_dict[column]),
                   whis='range', 
                   showfliers=True,
                   order=INCOME_GROUP_ORDER,
                   palette=[LOW_MID_COLOUR, UP_MID_COLOUR, HIGH_COLOUR],
                   boxprops=dict(alpha=ALPHA))
    plt.yticks(np.arange(0, ymax, 1.0))
    ax.set_yticklabels([*encoding_dict[column].keys()])
    ax.set_title(title)
    ax.set_ylabel(column)
    plt.show()
    
def violinplot_subplot_income_group(df, title, column):
    """Plot given list of columns by income group as violin plots."""
    num_plots = len(column)
    num_cols = 2
    num_rows = int(math.ceil(num_plots/2.0)) 
    f = plt.figure(figsize=(15, 5*num_rows))
    for i in range(num_plots):
        ymax = df[column[i]].map(encoding_dict[column[i]]).max()+1
        f.add_subplot(num_rows, num_cols, i+1)  
        ax = sns.violinplot(x=df['Income group'], 
                       y=df[column[i]].map(encoding_dict[column[i]]),
                       order=INCOME_GROUP_ORDER,
                      palette=[LOW_MID_COLOUR, UP_MID_COLOUR, HIGH_COLOUR])
        plt.setp(ax.collections, alpha=ALPHA)
        plt.yticks(np.arange(0, ymax, 1.0))
        ax.set_yticklabels([*encoding_dict[column[i]].keys()])
        ax.set_title(title[i])
    plt.tight_layout();
    plt.show()
   
    
def plot_split_income_group_ratios(df, title, column, xlabel_rotation=45):
    """
    Plot given column by income group as bar plots 
    with ratios of count of each value to total number of responses on the y-axis.
    """
    plt.figure(figsize=(6, 4))
    cols = [col for col in df.columns if column+'_Part_' in col]
    order = [*encoding_dict[column].keys()]
    xlabels = [*encoding_dict[column].values()]

    agg = responses[cols+['Income group']].groupby(['Income group'], sort=False).count()
    size = responses[cols+['Income group']].groupby(['Income group'], sort=False).size()
    agg['total'] = size
    for col in cols:
        agg[col] = agg[col]/agg['total']
        
    agg = pd.melt(agg.reset_index(), id_vars='Income group', value_vars=cols)
    ax = sns.barplot(data=agg,
                 x='variable',
                 y='value',
                 hue='Income group',
                 order=order,
                 hue_order=INCOME_GROUP_ORDER,
                 palette=[LOW_MID_COLOUR, UP_MID_COLOUR, HIGH_COLOUR],
                 alpha=ALPHA)
    plt.title(title, fontsize=12)
    ax.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')
    ax.set_ylabel('ratio of count to total responses')
    
    plt.show()
    
def plot_subplot_split_income_group_ratios(df, title, column, xlabel_rotation=45):
    """
    Plot given list of columns by income group as bar plots 
    with ratios of count of each value to total number of responses on the y-axis.
    """
    num_plots = len(column)
    num_cols = 3
    num_rows = int(math.ceil(num_plots/num_cols)) 
    f = plt.figure(figsize=(15, 5*num_rows))
    for i in range(num_plots):
        f.add_subplot(num_rows, num_cols, i+1)  
        
        cols = [col for col in df.columns if column[i]+'_Part_' in col]
        order = [*encoding_dict[column[i]].keys()]
        xlabels = [*encoding_dict[column[i]].values()]

        agg = responses[cols+['Income group']].groupby(['Income group'], sort=False).count()
        size = responses[cols+['Income group']].groupby(['Income group'], sort=False).size()
        agg['total'] = size
        for col in cols:
            agg[col] = agg[col]/agg['total']

        agg = pd.melt(agg.reset_index(), id_vars='Income group', value_vars=cols)
        ax = sns.barplot(data=agg,
                     x='variable',
                     y='value',
                     hue='Income group',
                     order=order,
                     hue_order=INCOME_GROUP_ORDER,
                     palette=[LOW_MID_COLOUR, UP_MID_COLOUR, HIGH_COLOUR],
                     alpha=ALPHA)
        plt.title(title[i], fontsize=12)
        ax.set_xticklabels(xlabels, rotation=xlabel_rotation)
        ax.set_xlabel('')
        if i%num_cols==0:
            ax.set_ylabel('ratio of x to total responses')
        else: 
            ax.set_ylabel('')
    plt.tight_layout();
    plt.show()
    
def plot_nominal_split_income_group_counts(df, title, column, xlabel_rotation=45):  
    """
    Plot given column by income group as bar plots 
    where the given column is the prefix of a question with multiple parts.
    """
    cols = [col for col in df.columns if column+'_Part_' in col]
    order = [*encoding_dict[column].keys()]
    xlabels = [*encoding_dict[column].values()]
    
    fig = plt.figure(figsize=(15,5))
        
    temp = pd.melt(df[df['Income group']=='Lower mid'][cols])
    ax = plt.subplot2grid((1,3),(0,0))
    l = sns.countplot(data=temp.loc[~temp['value'].isnull()], x='variable', 
                      order=order, 
                      color=LOW_MID_COLOUR,
                      alpha=ALPHA)
    plt.title(f'{title} - Lower-mid income', fontsize=12)
    l.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')
    
    temp = pd.melt(df[df['Income group']=='Upper mid'][cols])
    ax = plt.subplot2grid((1,3),(0,1))
    m = sns.countplot(data=temp.loc[~temp['value'].isnull()], x='variable', 
                      order=order, 
                      color=UP_MID_COLOUR,
                      alpha=ALPHA)
    plt.title(f'{title} - Upper-mid income', fontsize=12)
    m.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')

    temp = pd.melt(df[df['Income group']=='High'][cols])
    ax = plt.subplot2grid((1,3),(0,2))
    h = sns.countplot(data=temp.loc[~temp['value'].isnull()], x='variable', 
                      order=order, 
                      color=HIGH_COLOUR,
                      alpha=ALPHA)
    plt.title(f'{title} - High income', fontsize=12)
    h.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')

    plt.tight_layout()
    plt.show()

def plot_nominal_income_group_counts(df, title, column, xlabel_rotation=45): 
    """ Plot given column by income group as bar plots. """
    order = [*encoding_dict[column].keys()]
    xlabels = [*encoding_dict[column].values()]
    
    fig = plt.figure(figsize=(15,5))
    ax = plt.subplot2grid((1,3),(0,0))
    l = sns.countplot(df[df['Income group']=='Lower mid'][column], 
                      order=order, 
                      color=LOW_MID_COLOUR,
                      alpha=ALPHA)
    plt.title(f'{title} - Lower-mid income', fontsize=12)
    l.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')
    
    ax = plt.subplot2grid((1,3),(0,1)) 
    m = sns.countplot(df[df['Income group']=='Upper mid'][column], 
                      order=order, 
                      color=UP_MID_COLOUR,
                      alpha=ALPHA)
    plt.title(f'{title} - Upper-mid income', fontsize=12)
    m.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')
    
    ax = plt.subplot2grid((1,3),(0,2))
    h = sns.countplot(df[df['Income group']=='High'][column], 
                      order=order, 
                      color=HIGH_COLOUR,
                      alpha=ALPHA)
    plt.title(f'{title} - High income', fontsize=12)
    h.set_xticklabels(xlabels, rotation=xlabel_rotation)
    ax.set_xlabel('')
    
    plt.tight_layout()
    plt.show()

def spider_subplot_income_group_counts_by_question_part(df, q1, q2, question_parts):
    """
    Plot given question_parts of q1 grouped by q2 question parts as spider plots.
    Where q1 and q2 are multi-part questions.
    """
    num_plots = len(question_parts)
    num_cols = 3
    num_rows = int(math.ceil(num_plots/3.0)) 
    fig = plt.figure(figsize=(15, 5*num_rows))    
    titles = [encoding_dict[q1][key] for key in question_parts]
    for i in range(num_plots):
        fig.add_subplot(num_rows, num_cols, i+1)  

        cols = [*encoding_dict[q2].keys()]
        labels = [*encoding_dict[q2].values()]

        i_df = df[df[q1] == question_parts[i]]
        
        agg = i_df[cols+['Income group']].groupby(['Income group'], sort=False).count()
        size = i_df[cols+['Income group']].groupby(['Income group'], sort=False).size()
        agg['total'] = size
        agg = pd.melt(agg.reset_index(), id_vars='Income group', value_vars=cols)

        angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
        angles=np.concatenate((angles,[angles[0]])) # close the plot

        low_stats = agg[agg['Income group']=='Lower mid']['value'].values
        low_stats=np.concatenate((low_stats,[low_stats[0]])) # close the plot

        mid_stats = agg[agg['Income group']=='Upper mid']['value'].values
        mid_stats=np.concatenate((mid_stats,[mid_stats[0]])) # close the plot

        high_stats = agg[agg['Income group']=='High']['value'].values
        high_stats=np.concatenate((high_stats,[high_stats[0]])) # close the plot

        ax = fig.add_subplot(num_rows, num_cols, i+1, polar=True)
        ax.plot(angles, low_stats, 'o-', color=LOW_MID_COLOUR, linewidth=2)
        ax.fill(angles, low_stats, color=LOW_MID_COLOUR, alpha=0.2)

        ax.plot(angles, mid_stats, 'o-', color=UP_MID_COLOUR, linewidth=2)
        ax.fill(angles, mid_stats, color=UP_MID_COLOUR, alpha=0.2)

        ax.plot(angles, high_stats, 'o-', color=HIGH_COLOUR, linewidth=2)
        ax.fill(angles, high_stats, color=HIGH_COLOUR, alpha=0.2)

        ax.set_thetagrids(angles * 180/np.pi, labels)
        ax.set_title(titles[i], fontsize=12, pad=15)
        ax.grid(True)
        
    plt.tight_layout()
    plt.show()
    
def spider_subplot_income_group_counts(df, title, column):
    """
    Plot given list of columns by income group as spider plots 
    where the given column is the prefix of a question with multiple parts.
    """
    num_plots = len(column)
    num_cols = 3
    num_rows = int(math.ceil(num_plots/3.0)) 
    fig = plt.figure(figsize=(15, 5*num_rows))
    for i in range(num_plots):
        fig.add_subplot(num_rows, num_cols, i+1)  
        
        cols = [col for col in df.columns if column[i]+'_Part_' in col]
        cols.remove(column[i]+'_Part_11') # Remove "None" responses (Part_11)
        labels = [*encoding_dict[column[i]].values()]
        labels.remove('None') # Remove "None responses"

        agg = df[cols+['Income group']].groupby(['Income group'], sort=False).count()
        size = df[cols+['Income group']].groupby(['Income group'], sort=False).size()
        agg['total'] = size
        agg = pd.melt(agg.reset_index(), id_vars='Income group', value_vars=cols)
        agg = agg[agg['variable'] != column[i]+'_Part_11'] # Remove none (Part_11)

        angles=np.linspace(0, 2*np.pi, len(labels), endpoint=False)
        angles=np.concatenate((angles,[angles[0]])) # close the plot

        low_stats = agg[agg['Income group']=='Lower mid']['value'].values
        low_stats=np.concatenate((low_stats,[low_stats[0]])) # close the plot

        mid_stats = agg[agg['Income group']=='Upper mid']['value'].values
        mid_stats=np.concatenate((mid_stats,[mid_stats[0]])) # close the plot

        high_stats = agg[agg['Income group']=='High']['value'].values
        high_stats=np.concatenate((high_stats,[high_stats[0]])) # close the plot

        ax = fig.add_subplot(num_rows, num_cols, i+1, polar=True)
        ax.plot(angles, low_stats, 'o-', color=LOW_MID_COLOUR, linewidth=2)
        ax.fill(angles, low_stats, color=LOW_MID_COLOUR, alpha=0.2)

        ax.plot(angles, mid_stats, 'o-', color=UP_MID_COLOUR, linewidth=2)
        ax.fill(angles, mid_stats, color=UP_MID_COLOUR, alpha=0.2)

        ax.plot(angles, high_stats, 'o-', color=HIGH_COLOUR, linewidth=2)
        ax.fill(angles, high_stats, color=HIGH_COLOUR, alpha=0.2)

        ax.set_thetagrids(angles * 180/np.pi, labels)
        ax.set_title(title[i], fontsize=12, pad=15)
        ax.grid(True)
        
    plt.tight_layout()
    plt.show()
    

# Load World Bank classifications
worldbank_class = load_worldbank_classification()

# Load survey results
responses = load_survey_responses(RESPONSES_19, worldbank_class)

In [ ]:
def build_map(df, filename):
    """Plot country income group classifications on a world map."""
    # Control color spread by assigning codes for each income group
    income_dict = {'Low':0, 'Lower mid':2, 'Upper mid':6, 'High':9}
    df['income_code'] = df['Income group'].map(income_dict)
    
    # Create the Folium map
    world_map = folium.Map(location=[-20, 30],
                           zoom_start = 2,
                           min_zoom = 2,
                           max_zoom = 4,
                           width=860, height=720)
    folium.TileLayer('cartodbpositron', min_zoom=2, max_zoom=3, width=860, height=720).add_to(world_map)
    folium.Choropleth(geo_data=WORLD_COUNTRIES_JSON, data=df,
             columns=['Code', 'income_code'],
             key_on='feature.id',
             hover_name='Economy',
             fill_color='RdBu', fill_opacity=0.7, line_opacity=0.4,
             threshold_scale=[0, 2, 4, 6, 8, 10],
             legend_name='Income group (low -> high)',
              ).add_to(world_map)
    # Save map
    world_map.save(filename)
    return world_map

# Build and display map
filename = 'income_group_map.html'
world_map = build_map(worldbank_class,  filename)
#IFrame(filename, width=900, height=690)
world_map

The first step towards splitting the respondents into their income groups was to join the data to the classificaitons provided by the World Bank. And when this was done **one group disappeared!** There are **ZERO** responents from the low income group! (red on the map above) *(2)

So despite the great stories we hear about efforts to support AI in developing countries, it seems likely that they have not yet extended their reach to the very poorest countries, which are mainly in Africa.

Nevertheless, we will proceed with the analysis and take the lower-middle income group as a proxy for the low income group and examine those responses in comparison to the other two groups. From here on the three groups that are analysed will be referred to as low, mid and high income groups for simplicity. But please keep in mind that there is a whole other group, the worst off, who do not have any survey respondents to represent them.

In [ ]:
lineplot_survey_summary(responses)

In [ ]:
responses['Income group'].value_counts(normalize=True,sort=False).mul(100)

Looking at the number of respondents from each group per survey year, we see an increase in percentage of low income respondents relative to mid and high income respondents. It is not clear why the overall number of responses to the Kaggle survey have dropped in 2019 but the increased responses from lower income countries despite this could be taken as a promising sign of the overall increase in data science engagement in these regions.

For this survey year, the make-up of low, mid and high income respondents is about 35%, 18% and 47% respectively. The country make-up of each income group is shown below.

In [ ]:
barplot_income_group_country_counts(responses)

India dominates the low income group, Brazil, Russia and China dominate the mid income group, and the US dominates the high income group. Note, however, that the GNI ranking of India falls around the middle of all listed low income countries. Similarly, the GNI rankings of Brazil, Russia and China fall around the middle of the listed mid income countries (see World Bank data). So we will assume for now they are representative enough to ignore their dominating effect in this study. 

The US is placed towards the top of the high income group, beaten only by Switzerland and Norway. So keep in mind that any findings for this high income group may be skewed towards higher ranking countries.

# Demographics
***
##  Gender
As expected, females are a minority on Kaggle, as they are in STEM professions all over the world. 

There appears to be no significant difference between the proportion of males and female respondents in each income group. The percentages of females is approximately 16.9%, 14,7% and 16.6% for the low, mid and high income groups respectively.

In [ ]:
pieplot_income_group_percentage(responses, 'Gender', 'Q2')

Interestingly, this goes against the findings from recent research, which showed a tendency for countries that have *less* gender equality (associated with lower income countries) to have *more* women in science and technology than their gender-progressive counterparts (richer countries) [10][11]. Their suggested explanation for this was that women in countries with higher gender inequality, mostly the developing countries, are simply seeking the clearest possible path to financial freedom, which is often through STEM professions.

In [ ]:
Image("../input/2019-kaggle-survey-challenge-images/women_among_stem_grads.png", width=600)

The figure above shows that, out of countries studied, Algeria, a country with one of the lowest gender equality scores on the Global Gender Gap Index has the highest percentage of women among STEM graduates, just above 40%. While highly gender equal countries, such as Finland and Norway have a much lower percentage of women among STEM graduates, just over 20%.

We can speculate that the difference observed compared to Kaggle respondents could be because Kaggle is seen as more of a learning platform or passion (or even obsession for some) rather than an inherent part of the profession. That is to say, there is a roughly equal percentage of women who are really passionate about data science and find their way to Kaggle across all income groups, regardless of how many actually choose the field as a profession.


##  Age, Education, Coding Experience, ML Experience and Salary
There is a big difference in age groups between the three income categories. The median age group of respondents for the low income category is the 22 to 24 year-olds. For the mid income group it is the 25 to 29 year-olds and the high income median age group is the 30 to 34 year-olds. 

There is a gradual shift towards older respondents when moving from low to high income groups. This is not unexpected, as the average age in developing countries tends to be lower than that of developed countries.

In [ ]:
violinplot_income_group(responses, 'Age', 'Q1')

In terms of education, most respondents from high and mid income countries have a master's degree, while the bachelor degree-holders make up the largest group for low income countries. The expected shift towards higher levels of education moving from low to high income groups can be seen from Kaggle respondents. 

It may be tempting to jump to the conclusion that, even among Kaggle respondents, belonging to a higher income group increases the likelihood of pursuing higher educaiton. However, correlation doesn't necessarily imply causation and it may be just as likely that the low age of low income group respondents simply means they have not yet completed their education and if we check back in a few years they will also have their Masters or PhD.

In fact, education, coding experience, ML experience and salary all display the same trend as age, with the younger respondents from low income countries having less education, coding experience, ML experience, lower salaries and an increase in all four factors when moving from low to high income countries.

In [ ]:
violinplot_subplot_income_group(responses, 
                                ['Education', 'Coding Experience', 'ML Experience', 'Salary'],
                                ['Q4', 'Q15', 'Q23', 'Q10'])

# Work
***
## Title and Work Activities
As expected, large numbers of low income group respondents are students. 

Interestingly, there are proportionally many more respondents holding the title "Data Scientist" as opposed to "Software Engineer" and various other titles when moving from low towards high income gorups.

In [ ]:
plot_nominal_income_group_counts(responses, 'Title', 'Q5', xlabel_rotation=90)

In [ ]:
plot_nominal_split_income_group_counts(responses, 'Work Activities', 'Q9', xlabel_rotation=90)

However, the work activities performed by respondents are similar across income groups, with data analysis being the main activity. Building machine learning prototypes shows a slight increase in proportion from low to high income groups but all other activities are relatively constant across the board. There could well be a discrepancy between titles favoured in various countries and the work associated with each title. (After all, what's in a name?)

For example, if we examine the spider plot of the "Software Engineer" responses to work activities, we can see that proportionately more respondents in the low income group report "research that advances the state of the art of machine learning" being an important part of their role than those in the high income group. We do not observe such differences between the income groups in the "Data Scientist" and "Research Scientist" responses about work activities. So it is quite possible that the "Data Scientist" or "Research Scientist" role has in some cases not (yet?) been created in low income countries although that type of work exists and is often being done by people who have the title of "Software Engineer".

In [ ]:
spider_subplot_income_group_counts_by_question_part(responses, 'Q5', 'Q9', 
                                                    ['Software Engineer', 'Data Scientist', 'Research Scientist'])

## Company Size, Data Science Team Size, Use of ML Methods and Spending on AI
Very small company sizes are prevalent in all income groups, denoting the thriving data science and ML startup scene. Low and high income countries also show a big number of very large companies with over 10,000 employees. There is no clear trend moving from low to high income countries, as the middle income group reports a smaller percentage of very large companies comapred to mid-sized ones.

All three income groups show similar break-downs of data science team size corresponding to company size, with many having either small data science teams or very large ones, as shown by the (bimodal-ish) shapes in each violin plot.

In [ ]:
violinplot_subplot_income_group(responses, ['Company Size', 'DS Team Size'], ['Q6', 'Q7'])

High income country repondents report a significantly higher percentage of companies with well-established use of ML methods in production compared to the low and mid income groups, most of whom are still at the "exploring" stage.

Spending on ML/Cloud products is accoringly higher in high income countries.

In [ ]:
violinplot_subplot_income_group(responses, ['Use of ML Methods', 'Spending on ML/Cloud Products'], ['Q8', 'Q11'])

It is not surprising that we have been observing bias in our AI systems, both SOTA systems developed as part of research efforts and productive systems. If the AI creators are mainly people from the high income countries, it is only natural that in the first instance they build systems tailored to their needs, often with adverse outcomes or side-effects for those who do not fit the norm. 

Here are just a few examples of the types of bias I am referring to:
- systems able to better recognise faces with light coloured skin, such as the UK’s passport program that went live in 2016 - a "virtual speed-line for white people" [12];
- software to identify future violent criminal threats with very low accuracy on black defendents, such as that used by the Americal Criminal Justice System where "black defendants in particular were pegged to be a 77 percent higher risk for committing future crimes than reality proved" [13]; and   
- medical diagnosis and treatment data based on studies predominantly comprised of white patients. A recent study of trials involving some 150,000 patients in 29 countries at five different time points over the past 21 years showed that the ethnic makeup of the trials was about 86 percent white. This has resulted in the same advice being given to all patients, which was optimal for white people but not for all others [14].
    
Not to mention all the gender biased recruitment systems, voice assistants, etc. out there.

Although we are seeing good work being done to overcome these biases, I believe that bias in AI will inherently  tend to most adversely affect the people who are not involved in developing the technology. And this is one of the major motivating factors for this story and why it is so important that the disadvantaged are not left behind in this AI revolution. It is more important to enable people to build AI systems themselves than to build the systems for them. (Teach a man to fish ...)

Here is a recent tweet from the president of Nigeria, echoing this sentiment.

In [ ]:
Image("../input/2019-kaggle-survey-challenge-images/muhammadu_buhari_09_10_2019.png", width=600)

# Languages, Methods and Tools
***
## Primary Tool used at Work / School to Analyze Data
Local development environments are favoured for data analysis by every income group. There is a trend as we move from low through to high income group respondents that goes away from using basic statistics software such as Microsoft Excel and Google Sheets, towards more advanced statistics and BI software.

In [ ]:
plot_nominal_income_group_counts(responses, 'Software Use at Work/School', 'Q14', xlabel_rotation=90)

## Favourite Data Science Media Sources, Course Platforms and IDEs
Kaggle is the favourite media source for data science topics for respondents of low and mid income country respondents, while high income country respondents have a slight preference towards blogs *(3). Going from low towards high income respondents, there is a slight trend away from the use of YouTube as a source of data science information, towards journal publications and podcasts.

In [ ]:
spider_subplot_income_group_counts(responses,
                                   ['Data Science Media', 'DS Course Platforms', 'IDE Usage'],
                                   ['Q12', 'Q13', 'Q16'])

Coursera leads the way for data science courses across the board. Moving from low to high income groups there is a trend towards preferring university courses to Kaggle and Udemy, although those are still very popular with all income groups. It is possibly the case that a university education is still simply inaccessible to many people from lower income groups. These other options, which are often free and much more accessible to everyone are a great alternative.

Jupyter is the most popular IDE across all income groups and there is an increased use of many other IDEs listed in the survey question when moving from low towards high income country respondents.

## Usage of Hosted Notebooks
Interestingly, the percentage of repondents who do not regularly use any hosted notebooks is the lowest among the low income country respondents, with only 15.9% respondents answering "None" to that question. The mid and high income groups reported 22.8% and 33.2% not regularly using any hosted notebooks respectively. 

We can hypothesize that this is due to lack of own resources and, hence, heavier reliance on hosted products, especially free products such as the Kaggle Notebooks. When private resources are available, which is likely the case amongst high income group respondents, then  they tend to be preferred due to better performance than the free hosted products.

In [ ]:
plot_nominal_split_income_group_counts(responses, 'Hosted Notebook Usage', 'Q17', xlabel_rotation=90)

## Languages Usage
Python still leads the way across the board in terms of language use. We see a small increase in usage of R and SQL moving from low towards high income countries, and a small decrease in usage of C, C++ and Java.

In [ ]:
plot_nominal_split_income_group_counts(responses, 'Language Uages', 'Q18', xlabel_rotation=90)

## Usage of Data Viz Tools
The usage of data visualization tools is fairly consistent across all income groups, with the high income group showing a small increase in usage of the less popular tools, such as Shiny, D3.js and Leaflet / Folium, compared to the other two income groups. In some cases there is an obvious link between language use and data visualization tool use, such as the use of Shiny, which is only available for R.

In [ ]:
plot_nominal_split_income_group_counts(responses, 'Data Viz Libraries', 'Q20', xlabel_rotation=90)

## Specialised Hardware Usage
There is no big difference in hardware usage between income groups and it is great to see that even TPUs are used by a similar percentage of respondents from each of the three income groups.

In [ ]:
plot_split_income_group_ratios(responses, 'Hardware Usage', 'Q21', xlabel_rotation=90)

TPUs (Tensor Processing Units) are Google's custom-developed application-specific integrated circuits used to speed up machine learning workloads. They are a relatively new (first announced in 2016) piece of hardware specially designed for neural network inference and accelerate the performance of linear algebra computation. So they minimize the training time of large, complex neural network models. 

The great part is, since 2018 the TPU can be used by anyone as part of the GCP $300 free credit. So there are no barriers for people with a basic computer and internet connection to try it out at least once. And we observe that there has been no less interest in trying out this new resource among low income country respondents compared to mid and high income country resondents. 

In [ ]:
violinplot_income_group(responses, 'TPU Usage', 'Q22')

In fact if we look closer at the exact numbers, the percentages of respondents who have used a TPU at least once are 22.6%, 18.6% and 17.4% for the low, mid and high income groups respectively. Kudos to the low income country repondants for showing so much curiosity and initiative to try out the TPU!

In [ ]:
pd.crosstab(responses['Income group'],responses['Q22']).apply(lambda r: r/r.sum(), axis=1)

## Usage of ML Algorithms, Tools And Frameworks
There is not much difference between the income groups in terms of ML algorithm and framework usage \*(6). 

In [ ]:
plot_subplot_split_income_group_ratios(responses, 
                                           ['ML Algorithm Usage', 'Automated ML Tool Usage', 'ML Frameworks'],
                                           ['Q24', 'Q25', 'Q28'],
                                          xlabel_rotation=90)

Interestingly, the low and mid income group respondents tend to use proportionately more automated ML tools than high income group respondents, especially for data augmentation and model selection. The percentage of respondents who selected "None" when asked which categories of ML tools they use on a regular basis (Q25) was 36.7%, 39.7% and 49.5% for  low, mid and high income groups respectively. 

Two possible explanations for this come to mind:

1.  The relative youth, present lower level of education attained and smaller budgets make it more advantageous for the low and mid income groups to use these pre-built automated tools, while the higher education, more experience and higher budgets of the high income group enables them to take more advantage specialised knowledge which is not present in automated tools. 
    
2.  The promising achievements of automated tools are relatively recent developments, even by the standards of this fast-moving field. The youth and curiosity of the lower income groups causes more of them to want to experiment with these latest tools (as with the case of the TPU), while the (relatively) older members of higher income groups prefer to stick with what they know well.

Possibly, it is a combination of both these interdependent reasons.

## Usage of Computer Vision and NLP Methods
Again, we observe that the low income group responded with the lowest levels of "None" when asked what Computer Vision and Natural Language Processing Methods they used regularly, although they do not necessarily have the highest levels of responses to many of the specific methods queried on. 

In [ ]:
plot_subplot_split_income_group_ratios(responses, 
                                           ['Computer Vision Methods', 'NLP Methods'],
                                           ['Q26', 'Q27'],
                                          xlabel_rotation=90)

Unsurprisingly, standard classification methods in computer vision are the most widely used among all groups and word embedding methods, a fundamental part of most NLP methods, are the most widely used NLP methods across all income groups.

# Cloud Platforms and Proudcts
***
Note that the following questions on use of various products were only asked to people who (among other standard criteria) reported that the amount of money spent at their work on ML or cloud products was more than $0. This causes the data size to be skewed even more towards the high income group.

Interestingly, as we have observed with hosted notebook usage and automated tool usage, the fraction of respondents who do not regularly use any cloud computing platforms is lowest for the low income group.

In [ ]:
# Remove respondents who were not asked the next questions about product use
responses_product_usage = responses[(responses['Q15']!='I have never written code') & (responses['Q11']!='$0 (USD)')]
responses_product_usage.shape[0]

plot_split_income_group_ratios(responses_product_usage, 'Cloud Computing Platforms', 'Q29', xlabel_rotation=90)

Amazon (AWS) is the favoured cloud platform of Kaggle respondents, followed closely by the Google Cloud Platform (GCP). Azure lags behind in third place and the rest of the offerings trail far behind across the board. This is interesting because when we look at market share of these three platforms worldwide, research says Amazon has 33% of the Cloud Infrastructure Services market share, with Microsoft second at 16% and Google third at 8% [15].

In [ ]:
Image("../input/2019-kaggle-survey-challenge-images/synergy_research_CIS_Q219_bw.jpg", width=620)

AWS naturally has the advantage of being the first to market with their cloud offerings but the fact that Kaggle respondents put Google at close second place is interesting. I can only specualte that it is due to Google being more open-source than the others and their well-known AI and Tensorflow expertise as well as the TPU hardware to support it.

The large dominance of AWS among high income group respondents compared to the relatively even playing field between the big three among the other respondents could be due to the early adoption among the high income countries. Those companies possibly moved at time when AWS may have been the only viable cloud provider, while the other countries may have had a wider choice available to them by the time they started investing in cloud platforms.

The specific cloud computing products used by respondents across the three income groups are very similar, seen by the similar shapes representing each income group in each spider plot below. In some cases they reflect the proportionately higher AWS usage of the high income group compared to the low income group, where for example Amazon Sage Maker is often used instead of Google's NLP or Vision offerings.

In [ ]:
spider_subplot_income_group_counts(responses_product_usage,
                                   ['Cloud Computing Platforms', 'Cloud Computing Products', 'Big Data Analytics Products',
                                    'ML Products', 'Automated ML Tools', 'Relation DB Products'],
                                   ['Q29', 'Q30', 'Q31', 'Q32', 'Q33', 'Q34'])

Could the relative proportionatly lower regular usage of cloud platforms among high income group respondents be due to stricter data privacy laws in those countries? Laws that prevent companies from storing data or even performing calculations using cloud platforms? Data privacy laws are certainly some of the strictest in the world in Europe and I have witnessed first hand, this being cited as the reason for not moving to the cloud in one of the largest employers in Germany (my home country for the past 5 years). 

This brings us to another motivating factor of this story, a point that is brought up at the NeurIPS 2019 ML4D (ML for the Developing World) workshop [16]: Multinational companies sometimes make the deliberate choice to deploy new technologies in countries with little regulation in order to experiment. 

One example of this are the findings of Chris Sandbrook, a senior lecturer in conservation leadership at the University of Cambridge. In the field of forest conservation, in which it is common to use drones or hidden cameras to monitor wildlife and tree health, he has seen evidence that local people spend less time in forests once they know that camera traps or drones are present, feeling compromised even if they are doing nothing wrong. A lack of transparency about the drones, the data they collect and what they do with it, can generate suspicions, fear and conspiracy theories, resulting in a negative impact on psychological wellbeing [17].

Indeed, a study published recently in Nature Machine Intelligence, mapping the global landscape of existing ethics guidelines for AI, was unable to find evidence of any representation of African and South-American countries independently from international or supranational organizations [18].

In [ ]:
Image("../input/2019-kaggle-survey-challenge-images/nature_geo_distribution_ethical_ai_guidelines.png", width=600)

It is clear that the more economically developed countries are shaping the AI ethics debate  more than others, which raises concerns about neglecting local knowledge, cultural pluralism and global fairness.

# Conclusion
***
"Today’s global inequality of opportunity means that what matters most for your living conditions is the good or bad luck of your place of birth." (Roser, 2019) [19] Of course, hard work and life choices matter, but they pale in comparison to the advantages or disadvantages that come with this single, utterly random factor. 

I approached this analysis hoping to discover any important differences separating lower from higher income group respondents in order to gain a clearer understanding of how big the gap between them in terms of data science development really is. 

Right at the beginning there was a spanner thrown in the works, as the survey responses showed that there were no responses from the lowest income group. This in itself says a lot about the gap between rich and poor - it is still huge. The poorest people are not on Kaggle because they are busy trying to meet basic human needs which most of us take for granted.

However, based on responses from the lower-middle income countries, we can conclude that this group of respondents is a growing young and curious group. They don't have as high a level of education as the rest but this may well be because they are mostly still studying. Accordingly, they also have less experience with coding and machine learning and earn less than the other income groups. They take more advantage of hosted products and automated tools but overall they still use very similar methods, products and platforms.

The data science / ML startup scene is rife across all income groups and there are also many large companies across all income groups. But the amount of corporate ML spending and the maturity of ML within companies in the high income group is significantly higher than in the other two income groups. This ties in to the issues of bias in AI tending to most adversely affect the people who are rarely in positions to develop the technology. And the gap is inflated by the subsequent lack of regulation and ethics guidelines around data and AI in the developing countries.

In conclusion, I hope that this report has brought to your attention some important issues about global fairness in the context of the AI revolution. Although there are many great initiatives attempting to make data science accessible more people, we still need to do much more to support people in developing countries. They are a group of people who are just as curious, passionate and capable as the privileged and should be given as much assistance and opportunities as possible, in order to enable them to join in this AI revolution and to create a fairer world for everyone.

# References
***
[1] MIT Technology Review (2019), The computing power needed to train AI is now rising seven times faster than ever before: https://www.technologyreview.com/s/614700/the-computing-power-needed-to-train-ai-is-now-rising-seven-times-faster-than-ever-before/

[2] Nature (2019), Russian ‘CRISPR-baby’ scientist has started editing genes in human eggs with goal of altering deaf gene: https://www.nature.com/articles/d41586-019-03018-0

[3] MIT Technology Review (2019), The world’s first Gattaca baby tests are finally here: https://www.technologyreview.com/s/614690/polygenic-score-ivf-embryo-dna-tests-genomic-prediction-gattaca/

[4] Harari, Y. (2018), 21 Lessons for the 21st Century. London: Penguin Random House UK.

[5]  MIT Technology Review (2019), The future of AI research is in Africa: https://www.technologyreview.com/s/613848/ai-africa-machine-learning-ibm-google/

[6] Google Blog (2018), Google AI in Ghana: https://www.blog.google/around-the-globe/google-africa/google-ai-ghana/

[7] Jabri, M. (2018), Africa : It's time for you to SHINE !: https://www.kaggle.com/mhajabri/africai

[8] World Bank (2019), World Bank Country and Lending Groups: https://datahelpdesk.worldbank.org/knowledgebase/articles/906519

[9] World Bank (2019), How does the World Bank classify countries?: https://datahelpdesk.worldbank.org/knowledgebase/articles/378834-how-does-the-world-bank-classify-countries
    
[10] Stoet, G. & Geary, D. (2018) The Gender-Equality Paradox in Science, Technology, Engineering, and Mathematics Education https://journals.sagepub.com/doi/abs/10.1177/0956797617741719?journalCode=pssa

[11] The Atlantic (2018), The More Gender Equality, the Fewer Women in STEM: https://www.theatlantic.com/science/archive/2018/02/the-more-gender-equality-the-fewer-women-in-stem/553592/

[12] The Next Web (2019), UK passport program uses AI to create a virtual speed-line for white people: https://thenextweb.com/artificial-intelligence/2019/10/09/uk-passport-program-uses-ai-to-create-a-virtual-speed-line-for-white-people/

[13] Wired (2019), An AI-Run World Needs to Better Reflect People of Color: https://www.wired.com/story/an-ai-run-world-needs-to-better-reflect-people-of-color/

[14] Scientific American (2019), Clinical Trials Have Far Too Little Racial and Ethnic Diversity: https://www.scientificamerican.com/article/clinical-trials-have-far-too-little-racial-and-ethnic-diversity/

[15] Synergy Research Group (2019), Cloud Service Spending Still Growing Almost 40% per Year; Half of it Won by Amazon & Microsoft: https://www.srgresearch.com/articles/cloud-service-spending-still-growing-almost-40-year-half-it-won-amazon-microsoft

[16] NeurIPS 2019 Workshop (2019), Challenges and Risks of ML4D: https://sites.google.com/view/ml4d/home

[17] Sandbrook, C. (2015), The social implications of using drones for biodiversity conservation: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4623858/

[18]  Jobin, A. et al. (2019), Nature Machine Intelligence, The global landscape of AI ethics guidelines: https://www.nature.com/articles/s42256-019-0088-2

[19] Roser, M., Our World in Data (2019), Global Economic Inequality: https://ourworldindata.org/global-economic-inequality
   
# Notes
***
1. My initial idea was to split countries into "developing" and "developed". I went to the official UN list of developing countries and found that countries like 'China', 'Republic of Korea', 'Singapore', and 'Taiwan' were listed as developing countries, which surprised me. I then discovered that there is no mechanism for countries to be reclassified as they move up to developed countries and they may take it upon themselves to initiate reclassification. However, many countries do not do this so that they can continue to enjoy the benefits that come with being classified as "developing", such as longer transition periods for implementing trade agreements and protection against emergency import restrictions. So it seemed like the official UN list would not be appropriate to use for this analysis and I instead searched for a list more representative of the current world situation, which can be found in the World Bank Country and Lending Groups list [8].
    
2. It is possible that there are respondents from low income groups but too few such that they have fallen into the "Other" country category and unfortunately cannot be further analysed here.

3. It is importnat to keep in mind the selection bias here. As the survey included only Kaggle users, we cannot make any sweeping statements about the popularity of Kaggle amongst data scientists in general from this.

4. I have favoured violin plots when dealing with ordinal data, for the compactness of being able to represent three density distributions clearly in a single plot. The shape represents the density estimate of the variables, not actual values.

5. Bar plots have been used to visualise most of the nominal data. In the cases where the different income groups showed quite different response distributions, the bar plots were kept separate to allow easier visualisation of different trends. The response counts are kept as raw counts despite the vastly different number of respondents per income group, as the y-axes scales, each (automatically) optimised to fill the same height, results in easily comparable plots.

6. Where the response distributions were very similar, all three income groups are shown on the same polt. In this case it was necessary to convert the numbers being plotted to ratios by dividing the number of responses for each question part by the total number of responses to the question. Note that due to the way in which the questions were asked (allowing selection of more than one answer) the values shown are not percentages. This explains for instance, why we observe that for the Usage of ML Frameworks, although the highest ratio of "None" responses were from the high income group, they also have the highest ratio of positive responses to more of the other selection options. This tells us that more respondents from this group selected multiple response options compared to other groups, ie: those from the higher income group who *do* use ML frameworks regularly tend to use a larger variety of frameworks regularly than those who *do* use ML frameworks from lower income groups.

7. In the case of nominal data where it was not important to show the "None" or certain other responses, those responses have been removed and the remaining data plotted as spider plots. This allows a clear illustration of the similarities (and in a few cases differences) between all income groups with a quick glance at the resulting shapes.